In [3]:
import pandas as pd
import numpy as np
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.linear_model import LinearRegression
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

In [4]:
lamb = 10**-3  # regularization
gamma = 10**-2  # learning rate
iterations = 1000

In [7]:

data = pd.read_csv('New_Delhi_reviews.csv')
data = data.to_dict()
pprint(data.keys())

reviews = [r for r in data['review_full'].values() if type(r) == str]
ratings = [data['rating_review'][k] for k in data['rating_review'].keys() if type(data['review_full'][k]) == str]

cv = CountVectorizer()
# cv = CountVectorizer(stop_words=stopwords.words('english'))
X = cv.fit_transform(reviews)
print(type(X), X.shape)
X = hstack((X, np.ones((X.shape[0], 1))))
Y = np.array(ratings)

dict_keys(['rating_review', 'review_full'])
<class 'scipy.sparse._csr.csr_matrix'> (147579, 85907)


In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

B = X_train.T @ Y_train

w = np.random.rand(X_train.shape[1])
w_old = np.random.rand(X_train.shape[1])

for i in range(iterations):
    w -= (gamma/X_train.shape[0]) * (X_train.T @ (X_train @ w) - B + lamb*w)
    if i % 100 == 0:
        print('Iteration {}: delta={}'.format(i, np.linalg.norm(w - w_old)))
    w_old[:] = w

print('---')
Y_pred = X_test @ w
print('MSE={}'.format(np.linalg.norm(Y_test - Y_pred) / Y_test.shape[0]))

Iteration 0: delta=119.81872647107522
Iteration 100: delta=0.022091769756447773
Iteration 200: delta=0.013760364391445679
Iteration 300: delta=0.010852114567688032
Iteration 400: delta=0.00921101945445504
Iteration 500: delta=0.008085702881894299
Iteration 600: delta=0.007245159971959257
Iteration 700: delta=0.006588767484888978
Iteration 800: delta=0.006060843269637131
Iteration 900: delta=0.0056263626878882336
---
MSE=0.021877432203135986
